## Дебеторка

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle

import sys
import warnings
warnings.filterwarnings('ignore')
from tqdm import tqdm

from sklearn.metrics import mean_absolute_error, mean_squared_error

import statsmodels.formula.api as smf
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs
from scipy.optimize import minimize
plt.style.use('bmh')
import statsmodels.tsa.vector_ar as stv

In [ ]:
from statsmodels.tsa.api import VAR
from statsmodels.tsa.stattools import grangercausalitytests
from statsmodels.tsa.vector_ar.vecm import coint_johansen

In [ ]:
def mape(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
def mae(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred)))

## Функция АДФ теста на стационарность

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), style='bmh'):
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))

        y.plot(ax=ts_ax)
        ts_ax.set_title('Time Series Analysis Plots')
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax, alpha=0.05)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax, alpha=0.05)
        print('-------Критерий Дики Фуллера с константой, но без тренда-------------')
        print("Критерий Дики-Фуллера: ADF={}".format(sm.tsa.stattools.adfuller(y,autolag='BIC')[0]))
        print("Критерий Дики-Фуллера: p-value={:.5f}".format(sm.tsa.stattools.adfuller(y,autolag='BIC')[1]))
        print()
        print('-------Критерий Дики Фуллера с константой и трендом -------------')
        print("Критерий Дики-Фуллера: ADF={}".format(sm.tsa.stattools.adfuller(y,autolag='BIC',regression='ct')[0]))
        print("Критерий Дики-Фуллера: p-value={:.5f}".format(sm.tsa.stattools.adfuller(y,autolag='BIC',regression='ct')[1]))
        print()
        print('-------Критерий Дики Фуллера без константы и тренда -------------')
        print("Критерий Дики-Фуллера: ADF={}".format(sm.tsa.stattools.adfuller(y,autolag='BIC',regression='nc')[0]))
        print("Критерий Дики-Фуллера: p-value={:.5f}".format(sm.tsa.stattools.adfuller(y,autolag='BIC',regression='nc')[1]))

        plt.tight_layout()
    return

### Импорт данных

In [ ]:
DZ1 = pd.read_excel(
    'DZ_1.xlsx', usecols = ("A,C, H:J"))
pd.to_datetime(DZ1['period_name'])
DZ1.set_index('period_name', inplace=True)

In [ ]:
DZ2 = pd.read_excel(
    'DZ_2.xlsx', usecols = ("A,C, H:J"))
pd.to_datetime(DZ2['period_name'])
DZ2.set_index('period_name', inplace=True)

### Удаляю пустые значения по сегментам

In [ ]:
DZ1_1 = DZ1.dropna(subset=['segment'])
DZ2_1 = DZ2.dropna(subset=['segment'])

In [ ]:
total = pd.concat([DZ2_1, DZ1_1])

### Выделяю только сегмент B2C

In [ ]:
B2C = total[total['segment'] == 'B2C']

### Подсчет разбивок по мр_кодам

In [ ]:
B2C['mr_code'].value_counts()

### Созданием отдельных дата-сетов под каждый мр_код

In [ ]:
B2C_11 = B2C[B2C['mr_code'] == '011XX']
B2C_13 = B2C[B2C['mr_code'] == '013XX']
B2C_14 = B2C[B2C['mr_code'] == '014XX']
B2C_15 = B2C[B2C['mr_code'] == '015XX']
B2C_12 = B2C[B2C['mr_code'] == '012XX']
B2C_16 = B2C[B2C['mr_code'] == '016XX']
B2C_17 = B2C[B2C['mr_code'] == '017XX']

###  Группировка по дате - получение ряда

In [ ]:
B2C_DZ_11 = B2C_11.groupby(['period_name']).sum()
B2C_DZ_12 = B2C_12.groupby(['period_name']).sum()
B2C_DZ_13 = B2C_13.groupby(['period_name']).sum()
B2C_DZ_14 = B2C_14.groupby(['period_name']).sum()
B2C_DZ_15 = B2C_15.groupby(['period_name']).sum()
B2C_DZ_16 = B2C_16.groupby(['period_name']).sum()
B2C_DZ_17 = B2C_17.groupby(['period_name']).sum()

In [ ]:
B2C_DZ_11['ТДЗ'][24:].plot(figsize=(15,10), label = 'Fact')
plt.title('Размер TДЗ Центр')

### Создание отдельных фреймов на ТДЗ и ПДЗ по мр_кодам

In [ ]:
TDZ = B2C_DZ_11['ТДЗ']
PDZ = B2C_DZ_11['ПДЗ']
TDZ_12 = B2C_DZ_12['ТДЗ']
PDZ_12 = B2C_DZ_12['ПДЗ']
TDZ_13 = B2C_DZ_13['ТДЗ']
PDZ_13 = B2C_DZ_13['ПДЗ']
TDZ_14 = B2C_DZ_14['ТДЗ']
PDZ_14 = B2C_DZ_14['ПДЗ']
TDZ_15 = B2C_DZ_15['ТДЗ']
PDZ_15 = B2C_DZ_15['ПДЗ']
TDZ_16 = B2C_DZ_16['ТДЗ']
PDZ_16 = B2C_DZ_16['ПДЗ']
TDZ_17 = B2C_DZ_17['ТДЗ']
PDZ_17 = B2C_DZ_17['ПДЗ']

### Тут использую данные только с января 2018 года

In [ ]:
PDZ_new = PDZ[PDZ.index > '2017-12-01']
TDZ_new = TDZ[TDZ.index > '2017-12-01']

PDZ_new_12 = PDZ_12[PDZ_12.index > '2017-12-01']
TDZ_new_12 = TDZ_12[TDZ_12.index > '2017-12-01']
PDZ_new_13 = PDZ_13[PDZ_13.index > '2017-12-01']
TDZ_new_13 = TDZ_13[TDZ_13.index > '2017-12-01']
PDZ_new_14 = PDZ_14[PDZ_14.index > '2017-12-01']
TDZ_new_14 = TDZ_14[TDZ_14.index > '2017-12-01']
PDZ_new_15 = PDZ_15[PDZ_15.index > '2017-12-01']
TDZ_new_15 = TDZ_15[TDZ_15.index > '2017-12-01']
PDZ_new_16 = PDZ_16[PDZ_16.index > '2017-12-01']
TDZ_new_16 = TDZ_16[TDZ_16.index > '2017-12-01']
PDZ_new_17 = PDZ_17[PDZ_17.index > '2017-12-01']
TDZ_new_17 = TDZ_17[TDZ_17.index > '2017-12-01']

### Добавляю на каждый мр_код к ПДЗ и ТДЗ три лага для тестирования гипотез о интегрируемости ряда

In [ ]:
B2C_DZ_11['ТДЗ_1'] = B2C_DZ_11['ТДЗ'][24:].shift(1).fillna(0)
B2C_DZ_11['ТДЗ_2'] = B2C_DZ_11['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_11['ТДЗ_3'] = B2C_DZ_11['ТДЗ'][24:].shift(3).fillna(0)
B2C_DZ_11['ПДЗ_1'] = B2C_DZ_11['ПДЗ'][24:].shift(1).fillna(0)
B2C_DZ_11['ПДЗ_2'] = B2C_DZ_11['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_11['ПДЗ_3'] = B2C_DZ_11['ТДЗ'][24:].shift(3).fillna(0)

B2C_DZ_12['ТДЗ_1'] = B2C_DZ_12['ТДЗ'][24:].shift(1).fillna(0)
B2C_DZ_12['ТДЗ_2'] = B2C_DZ_12['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_12['ТДЗ_3'] = B2C_DZ_12['ТДЗ'][24:].shift(3).fillna(0)
B2C_DZ_12['ПДЗ_1'] = B2C_DZ_12['ПДЗ'][24:].shift(1).fillna(0)
B2C_DZ_12['ПДЗ_2'] = B2C_DZ_12['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_12['ПДЗ_3'] = B2C_DZ_12['ТДЗ'][24:].shift(3).fillna(0)

B2C_DZ_13['ТДЗ_1'] = B2C_DZ_13['ТДЗ'][24:].shift(1).fillna(0)
B2C_DZ_13['ТДЗ_2'] = B2C_DZ_13['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_13['ТДЗ_3'] = B2C_DZ_13['ТДЗ'][24:].shift(3).fillna(0)
B2C_DZ_13['ПДЗ_1'] = B2C_DZ_13['ПДЗ'][24:].shift(1).fillna(0)
B2C_DZ_13['ПДЗ_2'] = B2C_DZ_13['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_13['ПДЗ_3'] = B2C_DZ_13['ТДЗ'][24:].shift(3).fillna(0)

B2C_DZ_14['ТДЗ_1'] = B2C_DZ_14['ТДЗ'][24:].shift(1).fillna(0)
B2C_DZ_14['ТДЗ_2'] = B2C_DZ_14['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_14['ТДЗ_3'] = B2C_DZ_14['ТДЗ'][24:].shift(3).fillna(0)
B2C_DZ_14['ПДЗ_1'] = B2C_DZ_14['ПДЗ'][24:].shift(1).fillna(0)
B2C_DZ_14['ПДЗ_2'] = B2C_DZ_14['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_14['ПДЗ_3'] = B2C_DZ_14['ТДЗ'][24:].shift(3).fillna(0)

B2C_DZ_15['ТДЗ_1'] = B2C_DZ_15['ТДЗ'][24:].shift(1).fillna(0)
B2C_DZ_15['ТДЗ_2'] = B2C_DZ_15['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_15['ТДЗ_3'] = B2C_DZ_15['ТДЗ'][24:].shift(3).fillna(0)
B2C_DZ_15['ПДЗ_1'] = B2C_DZ_15['ПДЗ'][24:].shift(1).fillna(0)
B2C_DZ_15['ПДЗ_2'] = B2C_DZ_15['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_15['ПДЗ_3'] = B2C_DZ_15['ТДЗ'][24:].shift(3).fillna(0)

B2C_DZ_16['ТДЗ_1'] = B2C_DZ_16['ТДЗ'][24:].shift(1).fillna(0)
B2C_DZ_16['ТДЗ_2'] = B2C_DZ_16['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_16['ТДЗ_3'] = B2C_DZ_16['ТДЗ'][24:].shift(3).fillna(0)
B2C_DZ_16['ПДЗ_1'] = B2C_DZ_16['ПДЗ'][24:].shift(1).fillna(0)
B2C_DZ_16['ПДЗ_2'] = B2C_DZ_16['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_16['ПДЗ_3'] = B2C_DZ_16['ТДЗ'][24:].shift(3).fillna(0)

B2C_DZ_17['ТДЗ_1'] = B2C_DZ_17['ТДЗ'][24:].shift(1).fillna(0)
B2C_DZ_17['ТДЗ_2'] = B2C_DZ_17['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_17['ТДЗ_3'] = B2C_DZ_17['ТДЗ'][24:].shift(3).fillna(0)
B2C_DZ_17['ПДЗ_1'] = B2C_DZ_17['ПДЗ'][24:].shift(1).fillna(0)
B2C_DZ_17['ПДЗ_2'] = B2C_DZ_17['ТДЗ'][24:].shift(2).fillna(0)
B2C_DZ_17['ПДЗ_3'] = B2C_DZ_17['ТДЗ'][24:].shift(3).fillna(0)


### Общие фреймы по каждому мр_коду

In [ ]:
DZ_new  = B2C_DZ_11[24:]

DZ_new_12  = B2C_DZ_12[24:]
DZ_new_13  = B2C_DZ_13[24:]
DZ_new_14  = B2C_DZ_14[24:]
DZ_new_15  = B2C_DZ_15[24:]
DZ_new_16  = B2C_DZ_16[24:]
DZ_new_17  = B2C_DZ_17[24:]

In [ ]:
DZ_new

# 11 МР КОД

## ПДЗ

In [ ]:
tsplot(PDZ_new, lags=12)

### По графикам PACF ACF либо AR(1) либо I(1)

### Построение AR(1) - модели

In [ ]:
ols = sm.OLS(PDZ_new, DZ_new[['ПДЗ_1']]).fit()
ols.summary()

### Коэффициент при первом лаге близок к единице, проверим гипотезу о равенстве единице, если так, то вощьмем ряд I(1)

In [ ]:
hyp = 'ПДЗ_1 = 1'
t_test = ols.t_test(hyp)
print(t_test)

### Гипотеза не отвергается

In [ ]:
tsplot(ols.resid, lags=12)

### Как и ожидалось - остатки стационарны

### Так как мы уже подобрали хорошую модель, интерпретировав лаговые значения через дифференциал, осталось немного улучшить модель, подобрав лаги на составляющие ошибки

In [ ]:
p = range(0,5)
d = 0
q = range(0,13)

Ps = range(0,4)
Ds = range(0,2)
Qs = range(0,4)

from itertools import product

parameters = product(q)
parameters_list = list(parameters)

In [ ]:
parameters_list

### Подбор ведем по информационному критерию Байеса-Шварца

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(PDZ_new, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [0,0],
                                        enforce_stationarity=False).fit()
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

### Уберем незначимые элементы, добавим тренд либо константу по необходимости

In [ ]:
model=sm.tsa.statespace.SARIMAX(PDZ_new, order=(0, 1, [1,2,4,5,6,9]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit()
model.summary()

## Прогноз на ПДЗ 11 мр_кода

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
PDZ_new.plot(figsize=(15,10), label = 'Fact')
pred.plot(style='r--', label = 'Predicted part')
plt.title('Факт и прогноз на ПДЗ Центр')
plt.legend()
plt.show()


In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(PDZ_new[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(PDZ_new[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:

print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(PDZ_new[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(PDZ_new[-1:].values)*100))

### Доверительный интервал на пронозы в  5%

In [ ]:
fcast = model.get_forecast(6)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int(alpha=0.1)['lower ПДЗ'].plot(figsize=(15,6), style = 'g--', label = 'Нижняя граница интервала')
fcast.conf_int(alpha=0.1)['upper ПДЗ'].plot(figsize=(15,6), style = 'g--', label = 'Верхняя граница интервала')
pred.plot(style='b--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ПДЗ Центр с интервалами')
plt.fill_between(fcast.conf_int(alpha=0.1).index, fcast.conf_int(alpha=0.1)['lower ПДЗ'], fcast.conf_int(alpha=0.1)['upper ПДЗ'], where=None, interpolate=True, step=None,animated = True, data=1, color='C7')
plt.show()

## ТДЗ

### Подбор модели

In [ ]:
tsplot(TDZ_new, lags=12)

### По ACF и PACF модель очень похожа на I(1) или AR(1)

### Посмотрим, что нам даст проверочная регрессия

In [ ]:
ols1 = sm.OLS(TDZ_new, DZ_new['ТДЗ_1']).fit()
ols1.summary()

### Коэффициент близок к единице, а значит модели I(1), проверим гипотезу о равенстве единице

In [ ]:
hyp = 'ТДЗ_1 = 1'
t_test = ols1.t_test(hyp)
print(t_test)

### Гипотеза не отвергается

In [ ]:
tsplot(ols1.resid, lags=12)

### Ряд ошибок получившейся модели стационарен

## Подбираем соответсвующую модель I(1) по информационному критерию

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(TDZ_new, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [0,0,0,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

## Подберем частную ARIMA по значимости

In [ ]:
model=sm.tsa.statespace.SARIMAX(TDZ_new, order=(0, 1, [1,7]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                trend = [1,0,0,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

In [ ]:
tsplot(model.resid, lags=12)

### Ошибки стационарны

In [ ]:
Y = TDZ_new[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('ТДЗ Центр')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(TDZ_new[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения TДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(TDZ_new[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(TDZ_new[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(TDZ_new[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(6)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int(alpha=0.1).round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int(alpha=0.1)['lower ТДЗ'].plot(figsize=(15,10), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int(alpha=0.1)['upper ТДЗ'].plot(figsize=(15,10), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r-', label = 'Прогноз')
plt.legend()
plt.rcParams.update({'font.size': 20})
plt.tick_params(axis='both', which='both', labelsize=20, colors='b',
               grid_color='b', grid_alpha=1, width =0.5, length = 6, labelbottom = True, grid_linewidth =0.5)
plt.title('Прогноз на ТДЗ Центр с интервалами')
plt.fill_between(fcast.conf_int(alpha=0.1).index, fcast.conf_int(alpha=0.1)['lower ТДЗ'], fcast.conf_int(alpha=0.1)['upper ТДЗ'], where=None, interpolate=True, step=None,animated = True, data=1)
plt.show()

### Далее для всех моделей будут схожие рассуждения, поэтому не буду останавливаться на комментариях, а буду разбивать на логические части: Основная часть - ПДЗ или ТДЗ, подчасти (подбор модели) - проверка модели на стационарность, проверка гипотезы I(1); подбор модели ARIMA; потсроение графиков прогноза

# 12 МР_КОД

## ПДЗ

### Подбор модели

In [ ]:
tsplot(PDZ_new_12, lags=12)

In [ ]:
model_12 = sm.OLS(PDZ_new_12, DZ_new_12['ПДЗ_1']).fit()
model_12.summary()

In [ ]:
hyp = 'ПДЗ_1 = 1'
t_test =model_12.t_test(hyp)
print(t_test)

In [ ]:
tsplot(model_12.resid, lags=12)

### Подбор модели ARIMA

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(PDZ_new_12, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [0,0,0,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(PDZ_new_12, order=(0, 1, [4,9]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0,0,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

### Построение графиков

In [ ]:
Y = PDZ_new_12[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('PDZ_mr_code_12')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(PDZ_new_12[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(PDZ_new_12[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(PDZ_new_12[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(PDZ_new_12[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ПДЗ 12 мр_код с интервалами')
plt.show()

## ТДЗ

### Построение модели

In [ ]:
model_12_2 = sm.OLS(TDZ_new_12, DZ_new_12['ТДЗ_1']).fit()
model_12_2.summary()


In [ ]:
hyp = 'ТДЗ_1 = 1'
t_test =model_12_2.t_test(hyp)
print(t_test)

In [ ]:
tsplot(model_12_2.resid, lags=12)

### Подбор модели ARIMA

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(TDZ_new_12, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0,0,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(TDZ_new_12, order=(0, 1, [4]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0,0,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

In [ ]:
tsplot(model.resid, lags=14)

### Графики

In [ ]:
Y = TDZ_new_12[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('TDZ_mr_code_12')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(TDZ_new_12[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(TDZ_new_12[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(TDZ_new_12[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(TDZ_new_12[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ТДЗ 12 мр_код с интервалами')
plt.show()

# 13 МР КОД

## ПДЗ

### Подбор модели

In [ ]:
tsplot(PDZ_new_13, lags=12)

In [ ]:
model = sm.OLS(PDZ_new_13, DZ_new_13['ПДЗ_1']).fit()
model.summary()

In [ ]:
hyp = 'ПДЗ_1 = 1'
t_test =model.t_test(hyp)
print(t_test)

In [ ]:
tsplot(model.resid, lags=12)

### Подбор ARIMA модели

In [ ]:


%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(PDZ_new_13, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [0,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(PDZ_new_13, order=(0, 1, [3,4,5,6,9]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

In [ ]:
tsplot(model.resid, lags=14)

### Графики

In [ ]:
Y = PDZ_new_13[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('PDZ_mr_code_13')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(PDZ_new_13[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(PDZ_new_13[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(PDZ_new_13[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(PDZ_new_13[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ПДЗ 13 мр_код с интервалами')
plt.show()

## ТДЗ

In [ ]:
tsplot(TDZ_new_13, lags=12)

In [ ]:
model = sm.OLS(TDZ_new_13, DZ_new_13['ТДЗ_1']).fit()
model.summary()

In [ ]:
hyp = 'ТДЗ_1 = 1'
t_test =model.t_test(hyp)
print(t_test)

### Подбор ARIMA модели

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(TDZ_new_13, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(TDZ_new_13, order=(0, 1, [2,5,6]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

In [ ]:
tsplot(model.resid,lags=12)

In [ ]:
Y = TDZ_new_13[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('TDZ_mr_code_13')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(TDZ_new_13[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(TDZ_new_13[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(TDZ_new_13[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(TDZ_new_13[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ТДЗ 13 мр_код с интервалами')
plt.show()

# 14 МР КОД

## ПДЗ

### Подбор модели

In [ ]:
tsplot(PDZ_new_14, lags=12)

In [ ]:
model = sm.OLS(PDZ_new_14, DZ_new_14['ПДЗ_1']).fit()
model.summary()

In [ ]:
hyp = 'ПДЗ_1 = 1'
t_test = model.t_test(hyp)
print(t_test)

### Подбор ARIMA модели

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(PDZ_new_14, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(PDZ_new_14, order=(0, 1, [3,7,8]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

In [ ]:
tsplot(model.resid, lags=12)

### Графики

In [ ]:
Y = PDZ_new_14[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('PDZ_mr_code_14')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(PDZ_new_14[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(PDZ_new_14[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(PDZ_new_14[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(PDZ_new_14[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ПДЗ 14 мр_код с интервалами')
plt.show()

## ТДЗ

### Подбор модели

In [ ]:
tsplot(TDZ_new_14, lags=12)

In [ ]:
model = sm.OLS(TDZ_new_14, DZ_new_14['ТДЗ_1']).fit()
model.summary()

In [ ]:
hyp = 'ТДЗ_1 = 1'
t_test = model.t_test(hyp)
print(t_test)

### Подбор модели ARIMA

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(TDZ_new_14, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [0,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(TDZ_new_14, order=(0, 1, [3,6,7,9]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

In [ ]:
tsplot(model.resid, lags=12)

### Графики

In [ ]:
Y = TDZ_new_14[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('TDZ_mr_code_14')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.3f} процента'.format(mape(TDZ_new_14[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.3f} процента'.format((-np.mean(TDZ_new_14[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(TDZ_new_14[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(TDZ_new_14[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ТДЗ 14 мр_код с интервалами')
plt.show()

# 15 МР КОД

## ПДЗ

### Подбор моделм

In [ ]:
tsplot(PDZ_new_15, lags=12)

In [ ]:
model = sm.OLS(PDZ_new_15, DZ_new_15['ПДЗ_1']).fit()
model.summary()

In [ ]:
hyp = 'ПДЗ_1 = 1'
t_test = model.t_test(hyp)
print(t_test)

### Подбор ARIMA модели

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(PDZ_new_15, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(PDZ_new_15, order=(0, 1, [1,2,5,6,7,8]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

### Графики

In [ ]:
Y = PDZ_new_15[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('PDZ_mr_code_15')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(PDZ_new_15[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(PDZ_new_15[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(PDZ_new_15[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(PDZ_new_15[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ПДЗ 15 мр_код с интервалами')
plt.show()

## ТДЗ

### Подбор модели

In [ ]:
tsplot(TDZ_new_15, lags=12)

In [ ]:
model = sm.OLS(TDZ_new_15, DZ_new_15['ТДЗ_1']).fit()
model.summary()

In [ ]:
hyp = 'ТДЗ_1 = 1'
t_test = model.t_test(hyp)
print(t_test)

### Подбор ARIMA модели

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(TDZ_new_15, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(TDZ_new_15, order=(0, 1, [1,3,6,8,9]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

In [ ]:
tsplot(model.resid, lags=12)

### Графики

In [ ]:
Y = TDZ_new_15[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('TDZ_mr_code_15')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(TDZ_new_15[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(TDZ_new_15[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(TDZ_new_15[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(TDZ_new_15[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ТДЗ 15 мр_код с интервалами')
plt.show()

# 16 МР КОД

## ПДЗ

### Подбор модели

In [ ]:
tsplot(PDZ_new_16, lags=12)

In [ ]:
model = sm.OLS(PDZ_new_16, DZ_new_16['ПДЗ_1']).fit()
model.summary()

In [ ]:
tsplot(model.resid, lags=12)

In [ ]:
hyp = 'ПДЗ_1 = 1'
t_test = model.t_test(hyp)
print(t_test)

### Подбор ARIMA модели

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(PDZ_new_16, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(PDZ_new_16, order=(0, 1, [1,6]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

In [ ]:
tsplot(model.resid, lags=12)

### Графики

In [ ]:
Y = PDZ_new_16[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('PDZ_mr_code_16')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(PDZ_new_16[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(PDZ_new_16[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(PDZ_new_16[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(PDZ_new_16[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ПДЗ 16 мр_код с интервалами')
plt.show()

## ТДЗ

### ТДЗ подбор модели

In [ ]:
tsplot(TDZ_new_16, lags=12)

In [ ]:
model = sm.OLS(TDZ_new_16, DZ_new_16['ТДЗ_1']).fit()
model.summary()

In [ ]:
hyp = 'ТДЗ_1 = 1'
t_test = model.t_test(hyp)
print(t_test)

### Подбор модели ARIMA

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(TDZ_new_16, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(TDZ_new_16, order=(0, 1, [5,9]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

In [ ]:
tsplot(model.resid, lags=12)

### Графики

In [ ]:
Y = TDZ_new_16[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('TDZ_mr_code_16')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(TDZ_new_16[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(TDZ_new_16[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(TDZ_new_16[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(TDZ_new_16[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ТДЗ 16 мр_код с интервалами')
plt.show()

# 17 МР КОД

## ПДЗ

### Подбор модели

In [ ]:
tsplot(PDZ_new_17, lags=12)

In [ ]:
model = sm.OLS(PDZ_new_17, DZ_new_17['ПДЗ_1']).fit()
model.summary()

In [ ]:
tsplot(model.resid, lags=12)

### Подбор ARIMA модели

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(PDZ_new_17, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(PDZ_new_17, order=(0, 1, [2,5,6,7] ), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

### Графики

In [ ]:
Y = PDZ_new_17[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('PDZ_mr_code_16')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(PDZ_new_17[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(PDZ_new_17[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(PDZ_new_17[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(PDZ_new_17[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ПДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ПДЗ 17 мр_код с интервалами')
plt.show()

## ТДЗ

### Подбор модели

In [ ]:
tsplot(TDZ_new_17, lags=12)

In [ ]:
model = sm.OLS(TDZ_new_17, DZ_new_17['ТДЗ_1']).fit()
model.summary()

In [ ]:
tsplot(model.resid, lags=12)

### Подбор модели ARIMA

In [ ]:
%%time
results = []
best_bic = float("inf")

for param in tqdm(parameters_list):
    try:
        model=sm.tsa.statespace.SARIMAX(TDZ_new_17, order=(0, 1, param[0]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,1],
                                        enforce_stationarity=False).fit(disp=-1)
    except ValueError:
        print('wrong parameters:', param)
        continue
    bic = model.bic

    if bic < best_bic and bic>-10000:
        best_model = model
        best_bic = bic
        best_param = param
        results.append([param, model.bic])

warnings.filterwarnings('default')

result_table = pd.DataFrame(results)
result_table.columns = ['parameters', 'bic']
print(result_table.sort_values(by = 'bic', ascending=True).head(10))

In [ ]:
best_model.summary()

In [ ]:
model=sm.tsa.statespace.SARIMAX(TDZ_new_17, order=(0, 1, [2,3,6,8]), 
                                        #seasonal_order=(param[3], param[4], param[5], 12)
                                        trend = [1,0],
                                        enforce_stationarity=False).fit(disp=-1)
model.summary()

### Графики

In [ ]:
Y = TDZ_new_17[3:]
Y_predict = model.predict('2019-04-01','2021-10-01',   typ='levels')
Y_predict = Y_predict[3:]
plt.figure(figsize=[20, 12])
plt.rc('font', size=15)
plt.title('TDZ_mr_code_17')
plt.plot(Y_predict, 'b:', label='прогноз')
plt.plot(Y, 'r', label='факт. значения')
plt.legend()
plt.show()

In [ ]:
print('MAPE прогнозной модели составила : составила {0:.2f} процента'.format(mape(TDZ_new_17[24:].values, model.predict('2020-01-01','2021-05-01').values)))
print('Прогноз среднего изменения ПДЗ на июль-сентябрь 2021 года к маю 2021 : составит {0:.2f} процента'.format((-np.mean(TDZ_new_17[-1:].values)
                                                                              +np.mean(model.predict('2021-07-01','2021-09-01').values))/np.mean(TDZ_new_17[-1:].values)*100))

In [ ]:
print('MAE прогнозной модели составила : составила {0:.2f} рублей'.format(mae(TDZ_new_17[24:].values, model.predict('2020-01-01','2021-05-01').values)))

In [ ]:
fcast = model.get_forecast(4)
print('Forecast:')
print(fcast.predicted_mean.round(3))
print('Confidence intervals:')
print(fcast.conf_int().round(3))

In [ ]:
pred = model.predict('2020-04-01','2021-10-01',  typ='levels')
fcast.conf_int()['lower ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Нижняя граница интервала')
fcast.conf_int()['upper ТДЗ'].plot(figsize=(15,6), style = 'b--', label = 'Верхняя граница интервала')
pred.plot(style='r--', label = 'Прогноз')
plt.legend()
plt.title('Прогноз на ТДЗ 16 мр_код с интервалами')
plt.show()

# Гипотеза о векторной связи рядов, далее будут подобраны VAR и VECM модели

## Создание обычных месячных сезонных переменных

In [ ]:
May = [1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0]
June = [0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0]
July = [0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0]
August = [0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0]
Sept = [0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1]
Oct = [0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0]
Nov = [0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0]
Dec = [0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0]
Jan = [0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0]
Feb = [0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0]
March = [0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0]
April = [0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0]
Month = [May, June,July, August, Sept, Oct,Nov,Dec,Jan,Feb,March]

In [ ]:
tsplot(DZ['ТДЗ'].diff().fillna(0), lags=10)

## Создание экзогенной трендовой переменной на количество наблюдений

In [ ]:
t = []
const = []
for i in range(0,100):
    t.append(i)
    const.append(1)



## Разделение на обучающцю и тестирующую выборки

In [ ]:
X_train, X_test = DZ_new[0:-4], DZ_new[-4:]

## Остационаривание ряда для VAR моделей

In [ ]:
X_train_log = np.log(X_train)
X_train_log_diff = X_train_log.diff().fillna(0)

In [ ]:
X_train_log_diff

In [ ]:
tsplot(X_train_log_diff['ТДЗ'], lags=12)

## Тест на причинность по Грейнджеру

In [ ]:
print(grangercausalitytests(X_train_log_diff[['ТДЗ', 'ПДЗ']], maxlag=10, addconst=True, verbose=True))

### Как можно увидеть, ряды ТДЗ и ПДЗ не порождают друг друга

### Теперь попробуем "насильно" построить VAR модель для двух рядов

## Выбираем оптимальное значение лага по информационным критериям

In [ ]:
model = VAR(endog=X_train_log_diff[['ТДЗ', 'ПДЗ']])
res = model.select_order(11)
res.summary()

### 11 лаг имеет лучшую статистику по информационным критериям

## Построение VAR модели на 11 лагах

In [ ]:
model_fit = model.fit(maxlags=11)
model_fit.summary()

### Ничего не значимо - модель плохая

### Также можно "наивно" посмотреть на эту модель со стороны меньшего количества лагов

In [ ]:
model_fit_naive = model.fit(maxlags=2)
model_fit_naive.summary()

### Результат тот же - ряды не порождают друг друга

## Посмотрим прогноз по полученным моделям

In [ ]:
lag_order = model_fit.k_ar
input_data = X_train_log_diff[['ПДЗ','ТДЗ']].values[-lag_order:]
print(input_data)
pred = model_fit.forecast(y=input_data, steps=4)
pred = (pd.DataFrame(pred, index=X_test[['ПДЗ', 'ТДЗ']].index, columns=X_test[['ПДЗ','ТДЗ']].columns + '_pred'))
print(pred)

## Обратное преобразование данных логарифма и разности

In [ ]:
def invert_transformation(X_train, pred_df):
    forecast = pred.copy()
    columns = ['ПДЗ','ТДЗ']
    for col in columns:
        forecast[str(col)+'_pred'] = np.log(X_train[col].iloc[-1]) + forecast[str(col) +'_pred'].cumsum()
    return forecast
output = invert_transformation(X_train[['ПДЗ','ТДЗ']], pred)
print(output)
output_original = np.exp(output)
print(output_original)

In [ ]:
plt.figure(figsize = (12,6))
plt.xlabel('Дата')
ax1 = X_test.ПДЗ.plot(color = 'blue', grid = True, label = 'Actual PDZ')
ax2 = output_original.ПДЗ_pred.plot(color = 'red', grid = True, label = 'Predicted PDZ')

ax1.legend(loc=1)
ax2.legend(loc=2)
plt.show()

In [ ]:
mape(X_test.ПДЗ, output_original.ПДЗ_pred)

### Имеется некоторое различие на тесте в 0.05 на 10^9

In [ ]:
plt.figure(figsize = (12,6))
plt.xlabel('Дата')
ax1 = X_test.ТДЗ.plot(color = 'blue', grid = True, label = 'Actual TDZ')
ax2 = output_original.ТДЗ_pred.plot(color = 'red', grid = True, label = 'Predicted TDZ')

ax1.legend(loc=1)
ax2.legend(loc=2)
plt.show()

### Различия плюс\минус тех же порядков

# Посмотрим на VECM модель

## найдем ранг коинтеграции рядов, устанавливая значение лага на k_VAR - 1 =10

In [ ]:
mod1 = coint_johansen(X_train[['ТДЗ','ПДЗ']],0,10)

### Функция для получения LR статистики через разложение собственных векторов (Eig) и следа матрицы (Trace)

In [ ]:
def joh_output(res):
    
    output = lr = pd.DataFrame([res.lr2,res.lr1], index = ['Maxeig', 'Trace'])
    print('\nCE w/ constant intercept\n', lr.T,'\n')
    print("Critical values(90%, 95%, 99%) of MaxEig\n", mod1.cvm,'\n')
    print('Critical values(90%, 95%, 99%) of Trace\n', mod1.cvt,'\n')

In [ ]:
joh_output(mod1)

### По тесту с константой можно наблюдать, что Ранг конинтеграции равен 1

In [ ]:
mod1 = coint_johansen(X_train[['ТДЗ','ПДЗ']],1,10)
joh_output(mod1)

### Отсутвтие коинтеграции  для трендовой составляющей

In [ ]:
mod1 = coint_johansen(X_train[['ТДЗ','ПДЗ']],-1,10)
joh_output(mod1)

### Тест без константы и тренла указывает на наличие конинтеграционного вектора

## Аналогисные результаты для автоматического подбора ранга коинтеграции

In [ ]:
vec_rank1 = stv.vecm.select_coint_rank(X_train[['ТДЗ','ПДЗ']], det_order =-1, k_ar_diff = 10, method = 'maxeig', signif=0.05)
print(vec_rank1.summary())

In [ ]:
vec_rank1.rank

### Получение предсказания

In [ ]:
vecm = stv.vecm.VECM(endog = X_train[['ТДЗ','ПДЗ']], k_ar_diff = 10, coint_rank = 1, deterministic = 'nc')
vecm_fit = vecm.fit()
vecm_fit.summary()


In [ ]:
vecm_fit.predict(steps=4)

### Создание доверительного интервала на прогнозы

In [ ]:
forecast, lower, upper = vecm_fit.predict(4, 0.05)
print("\lower bounds of confidence intervals:")
print(lower.round(3))
print("\npoint forecasts:")
print(forecast.round(3))
print("\nupper bounds of confidence intervals:")
print(upper.round(3))

# Графики сравнения ПДЗ и ТДЗ прогноза с тестом

In [ ]:
forc = forecast[:,1]
newforc = np.squeeze(forc)
plt.figure(figsize = (12,6))
plt.plot(newforc, color='green', marker='o', linestyle='dashed',
    linewidth=2, markersize=12, label ='predicted PDZ')
plt.plot(X_test.ПДЗ.reset_index().drop(columns = 'period_name'), color='red', marker='o', linestyle='dashed',
 linewidth=2, markersize=12, label = 'test PDZ')
plt.legend()
plt.show()

In [ ]:
mape(newforc,X_test.ПДЗ)

In [ ]:
forc = forecast[:,0]
newforc = np.squeeze(forc)
plt.figure(figsize = (12,6))
plt.plot(newforc, color='green', marker='o', linestyle='dashed',
    linewidth=2, markersize=12, label ='predicted TDZ')
plt.plot(X_test.ТДЗ.reset_index().drop(columns = 'period_name'), color='red', marker='o', linestyle='dashed',
 linewidth=2, markersize=12, label = 'test TDZ')
plt.legend()
plt.show()

In [ ]:
mape(newforc,X_test.ТДЗ)

# Графики прогноза на 5% доверительном интервале

In [ ]:
vecm_fit.plot_forecast(steps = 4, n_last_obs=10)

In [ ]:
#cred['balance_boxcox'], lambd = scs.boxcox(cred.balance)
#print('Преобразование Бокса-Кокса с параметром lambda = {}'.format(lambd))
#tsplot(cred.balance_boxcox, lags=36)
#cred['balance_delta'] = cred.balance_boxcox - cred.balance_boxcox.shift(1)
#tsplot(cred.balance_delta[1:], lags=36)
#def invboxcox(y,lmbda):
    # обрабтное преобразование Бокса-Кокса
    #if lmbda == 0:
        #return(np.exp(y))
    #else:
        #return(np.exp(np.log(lmbda*y+1)/lmbda))